<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/sglogo.png width=50  align="left">

# Database Engine - MariaDB/MySQL Dynamic Credentials
---
Dynamic Secrets are generated when they are accessed.  In other words, Dynamic Secrets do not exist until they are read.
Vault has built-in revocation mechanism.
Database Secrets engine:
* Cassandra
* Couchbase
* Elasticsearch
* InfluxDB
* HanaDB
* MongoDB
* MongoDB Atlas
* MSSQL
* MySQL/MariaDB
* PostgreSQL
* Oracle
* Redshift

## Business Value of Capability
We use dynamic secrets to allow applications to get dynamic creds to access databases.  
<img src=images/db-secret-engine.png width=800>

### Setup   
```
vrd1
```   

#### **Note:** 
* Run > Render All Markdown Cells
* Edit > Clear all outputs
---

In [13]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [2]:
unset VAULT_TOKEN; vault login root

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                root
token_accessor       dRq6usE3NFp7vjYtgtH8uqQv
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]


In [ ]:
vault status; vault version

**Create a database**  
Note the additional options MYSQL_USER and MYSQL_PASSWORD are being used to create demodb, it's not for Vault.


In [15]:
## NO LONGER IN USE
docker create \
  --name=mariadb \
  -e PUID=1000 \
  -e PGID=1000 \
  -e TZ=Asia/Singapore \
  -e MYSQL_ROOT_PASSWORD=password \
  -e MYSQL_DATABASE=vaultdemodb `#optional` \
  -e MYSQL_USER=tio `#optional` \
  -e MYSQL_PASSWORD=password `#optional` \
  -p 3306:3306 \
  --restart unless-stopped \
  linuxserver/mariadb

eccb0a5df7c7cc5577be09976cd31878da431e7254f081850bae280d272a45fb


**Start the database**

In [16]:
#  -v /tmp/workshop-data:/var/lib/mysql \
docker run --rm --name workshop-mysql \
  -p 3306:3306 \
  -e MYSQL_ROOT_PASSWORD=root \
  -e MYSQL_ROOT_HOST=% \
  -e MYSQL_DATABASE=my_app \
  -e MYSQL_USER=vault \
  -e MYSQL_PASSWORD=vaultpw \
  -d mysql/mysql-server:5.7.21

e6d0695ac94e18af1811541aa8983279d22470b2b106b996ef4070ba5cc7618b


In [17]:
# mariadb
docker ps

CONTAINER ID   IMAGE                       COMMAND                  CREATED         STATUS                            PORTS                               NAMES
e6d0695ac94e   mysql/mysql-server:5.7.21   "/entrypoint.sh mysq…"   8 seconds ago   Up 7 seconds (health: starting)   0.0.0.0:3306->3306/tcp, 33060/tcp   workshop-mysql
9b8d32ee26c5   nginx:1.13.6-alpine         "nginx -g 'daemon of…"   2 weeks ago     Up 22 minutes                     80/tcp, 0.0.0.0:8443->443/tcp       nginx_proxy
1124a54fb16e   cyberark/conjur             "conjurctl server"       2 weeks ago     Up 26 seconds                     80/tcp                              conjur_server
3f4eef754b3c   kindest/node:v1.20.2        "/usr/local/bin/entr…"   3 weeks ago     Up 22 minutes                     127.0.0.1:51133->6443/tcp           terraform-learn-control-plane


In [18]:
docker exec -it workshop-mysql mysql -u root -p 'root' \
  -e "GRANT ALL PRIVILEGES ON *.* TO 'vaultadmin'@'%' IDENTIFIED BY 'vadmin' WITH GRANT OPTION;  FLUSH PRIVILEGES;"

mysql: [Warning] Using a password on the command line interface can be insecure.


In [1]:
docker exec -it workshop-mysql mysql -u root -p'root' \
  -e "show grants for root; show grants for vaultadmin"

mysql: [Warning] Using a password on the command line interface can be insecure.
+-------------------------------------------------------------+
| Grants for root@%                                           |
+-------------------------------------------------------------+
| GRANT ALL PRIVILEGES ON *.* TO 'root'@'%' WITH GRANT OPTION |
| GRANT PROXY ON ''@'' TO 'root'@'%' WITH GRANT OPTION        |
+-------------------------------------------------------------+
+-------------------------------------------------------------------+
| Grants for vaultadmin@%                                           |
+-------------------------------------------------------------------+
| GRANT ALL PRIVILEGES ON *.* TO 'vaultadmin'@'%' WITH GRANT OPTION |
+-------------------------------------------------------------------+


**Display existing database users**

In [2]:
docker exec -it workshop-mysql mysql -u vaultadmin -p'vadmin' -e "select user from mysql.user;"

mysql: [Warning] Using a password on the command line interface can be insecure.
+---------------+
| user          |
+---------------+
| root          |
| vault         |
| vaultadmin    |
| healthchecker |
| mysql.session |
| mysql.sys     |
| root          |
+---------------+


In [ ]:
docker exec -it workshop-mysql mysql -u vaultadmin -p'vadmin' -e "show databases"

In [7]:
docker exec -it workshop-mysql mysql -u vaultadmin -p'vadmin' -e "use my_app; create table test (score int not null)"

mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1050 (42S01) at line 1: Table 'test' already exists


In [4]:
docker exec -it workshop-mysql mysql -u vaultadmin -p'vadmin' -e "use my_app; insert into test values (10)"

mysql: [Warning] Using a password on the command line interface can be insecure.


In [5]:
docker exec -it workshop-mysql mysql -u vaultadmin -p'vadmin' -e "use my_app; select * from test"

mysql: [Warning] Using a password on the command line interface can be insecure.
+-------+
| score |
+-------+
|    10 |
+-------+


**Password Policy**

In [8]:
tee /tmp/example_policy.hcl <<EOF
length=8

rule "charset" {
  charset = "abcdefghijklmnopqrstuvwxyz"
  min-chars = 1
}

rule "charset" {
  charset = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
  min-chars = 1
}

rule "charset" {
  charset = "0123456789"
  min-chars = 1
}

rule "charset" {
  charset = "!@#$%^&*"
  min-chars = 1
}
EOF

length=8

rule "charset" {
  charset = "abcdefghijklmnopqrstuvwxyz"
  min-chars = 1
}

rule "charset" {
  charset = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
  min-chars = 1
}

rule "charset" {
  charset = "0123456789"
  min-chars = 1
}

rule "charset" {
  charset = "!@#$%^&*"
  min-chars = 1
}


In [9]:
vault write sys/policies/password/example policy=@/tmp/example_policy.hcl

Error writing data to sys/policies/password/example: Put "https://127.0.0.1:8200/v1/sys/policies/password/example": http: server gave HTTP response to HTTPS client


In [10]:
vault read sys/policies/password/example/generate

Error reading sys/policies/password/example/generate: Get "https://127.0.0.1:8200/v1/sys/policies/password/example/generate": http: server gave HTTP response to HTTPS client


### Demo
---

### Enable Database secrets engine

In [11]:
vault secrets enable -path=mdb1 -default-lease-ttl=5m -max-lease-ttl=30m database

Error enabling: Post "https://127.0.0.1:8200/v1/sys/mounts/mdb1": http: server gave HTTP response to HTTPS client


**Configure it with the correct plugin, connection string and allowed_roles**

You can specify `root_rotation_statements` when writing to the path <database/config/<connection>.  For example
```  
   root_rotation-statements="ALTER USER {{username}}'@'localhost' IDENTIFIED BY '{{password}}';"
```  
    

In [14]:
vault write mdb1/config/mysql \
    plugin_name=mysql-legacy-database-plugin \
    connection_url="vaultadmin:vadmin@tcp(127.0.0.1:3306)/" \
    allowed_roles="manager,engineer" \
    root_rotation-statements="ALTER USER '{{username}}'@'%' IDENTIFIED BY '{{password}}';" \
    password_policy="example" \
    username="vaultadmin" \
    password="vadmin" 

**Create one or more Vault roles with TTLs and SQL that specify permissions**

In [18]:
# read only
vault write mdb1/roles/manager \
    db_name=mysql \
    creation_statements="CREATE USER '{{name}}'@'%' IDENTIFIED BY '{{password}}';GRANT SELECT ON my_app.* TO '{{name}}'@'%';" \
    default_ttl="3m" \
    metadata="engineering manager SG"
    max_ttl="24h"

Success! Data written to: mdb1/roles/manager


In [16]:
# grant all
vault write mdb1/roles/engineer \
    db_name=mysql \
    creation_statements="CREATE USER '{{name}}'@'%' IDENTIFIED BY '{{password}}';GRANT ALL ON my_app.* TO '{{name}}'@'%';" \
    default_ttl="10m" \
    max_ttl="24h"

Success! Data written to: mdb1/roles/engineer


In [ ]:
# OPTIONAL: rotate root password
#vault write -force mdb1/rotate-root/mysql

**Note**: When this is done, the password for the user specified in the previous step is no longer accessible. Because of this, it is highly recommended that a user is created specifically for Vault to use to manage database users.

## Use Case: Get Dyamic Database credential

In [19]:
vault read mdb1/creds/manager

Key                Value
---                -----
lease_id           mdb1/creds/manager/XISAo12MwZEpuUwhG7FPt9ou
lease_duration     3m
lease_renewable    true
password           6Jm@l5*w
username           v_mana_X6rWJAzIe


In [3]:
#-output-curl-string always available
vault read mdb1/creds/engineer

Error reading mdb1/creds/engineer: Error making API request.

URL: GET http://127.0.0.1:8200/v1/mdb1/creds/engineer
Code: 500. Errors:

* 1 error occurred:
	* dial tcp 127.0.0.1:3306: connect: connection refused




&nbsp;

   
**Display database users** (note new user added by Vault)

In [ ]:
docker exec -it workshop-mysql mysql -u vaultadmin -p'vadmin' -e "select user from mysql.user"; 

**Database test using the newly created credential**

In [ ]:
docker exec -it workshop-mysql mysql -u v_engi_qVrNwPIBZ -p'SQ&U28vx' \
  -e "use my_app; select * from my_app.test"

In [ ]:
# manager - read only role
docker exec -it workshop-mysql mysql -u v_engi_qVrNwPIBZ -p'SQ&U28vx' \
  -e "use my_app ; insert into test values (7)"

In [ ]:
docker exec -it workshop-mysql mysql -u v_engi_qVrNwPIBZ -p'SQ&U28vx' \
  -e "use my_app; select * from my_app.test"

**List the Vault leases**

In [ ]:
vault list sys/leases/lookup/mdb1/creds/engineer/

**Details on a lease, e.g. the remaining lifetime of the credential**  
COPY LEASE ABOVE TO REPLACE END OF THIS LINE 

In [ ]:
vault write sys/leases/lookup lease_id=mdb1/creds/engineer/5ifXuzH5W7WAvV91KGP2BcKr

**Renew leases*

In [ ]:
vault write sys/leases/renew lease_id=mdb1/creds/engineer/nsAyHW7BzHhRVSRvt7R1G2Ak

**Revoke a specific Vault lease**   
Vault automatically deletes expired credentials but if credentials are compromised, you can revoke them immediately.

In [ ]:
vault write sys/leases/revoke lease_id=mdb1/creds/engineer/nsAyHW7BzHhRVSRvt7R1G2Ak

In [ ]:
vault lease revoke mdb1/creds/engineer/nsAyHW7BzHhRVSRvt7R1G2Ak

**Revoke all Vault leases**

In [ ]:
vault lease revoke -prefix=true mdb1/creds/engineer

**Disable Database Secret Engine**

In [ ]:
vault secrets disable mdb1/ 

---
### Cleanup


**Shutdown Docker**

In [4]:
docker stop workshop-mysql

workshop-mysql


**Remove the container**

In [5]:
docker rm workshop-mysql

workshop-mysql


**TODO:**. 
- root-rotation
- wrapping token

**END**

In [ ]:
docker exec -it mariadb mysql -u root -p'password' \
  -e "GRANT ALL PRIVILEGES ON *.* TO 'vaultadmin'@'%' IDENTIFIED BY 'vadmin' WITH GRANT OPTION; FLUSH PRIVILEGES;"